In [11]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [15]:
df = pd.read_csv('https://raw.githubusercontent.com/vincentarelbundock/Rdatasets/master/csv/DAAG/spam7.csv')

df.drop('Unnamed: 0', axis=1, inplace=True)
y = df.yesno.apply(lambda x: 0 if x == 'y' else 1)

df.drop('yesno', axis=1, inplace=True)

for i in range(0,6):
    for j in range(i+1,6):
        df[str(i)+str(j)] = df.iloc[:,i] * df.iloc[:,j]
X = df.iloc[:,:-1]

In [28]:
def AdaBoost_scratch(X,y, M=10, learning_rate = 1):
    # инициалиазция служебных переменных
    N = len(y)
    estimator_list, y_predict_list, estimator_error_list, estimator_weight_list, sample_weight_list = [],[],[],[],[]

    # инициализация весов
    sample_weight = np.ones(N) / N
    sample_weight_list.append(sample_weight.copy())

    # цикл по длине М
    for m in range(M):   

        # обучим базовую модель и получим предсказание
        estimator = DecisionTreeClassifier(max_depth = 1, max_leaf_nodes=2)
        estimator.fit(X, y, sample_weight=sample_weight)
        y_predict = estimator.predict(X)

        # Маска для ошибок классификации
        incorrect = (y_predict != y)

        # Оцениваем ошибку
        weight_error = 0
        weight_summ = 0
        #for i in range(N):
        weight_error = np.sum(sample_weight * incorrect)
        weight_summ = np.sum(sample_weight)    
        estimator_error = weight_error / weight_summ
        
        print(weight_error, weight_summ, estimator_error)
        # Вычисляем вес нового алгоритма
        estimator_weight = learning_rate * np.log((1-estimator_error)/estimator_error)
        # Получаем новые веса объектов
        sample_weight *= np.exp(estimator_weight * incorrect * ((sample_weight > 0) | (estimator_weight < 0)))
        print(sample_weight.value_counts())
        # Сохраяем результаты данной итерации
        estimator_list.append(estimator)
        y_predict_list.append(y_predict.copy())
        estimator_error_list.append(estimator_error.copy())
        estimator_weight_list.append(estimator_weight.copy())
        sample_weight_list.append(sample_weight.copy())
        


    # Для удобства переведем в numpy.array   
    estimator_list = np.asarray(estimator_list)
    y_predict_list = np.asarray(y_predict_list)
    estimator_error_list = np.asarray(estimator_error_list)
    estimator_weight_list = np.asarray(estimator_weight_list)
    sample_weight_list = np.asarray(sample_weight_list)
    # Получим предсказания
    preds = (np.array([np.sign((y_predict_list[:,point] * estimator_weight_list).sum()) for point in range(N)]))
    print('Accuracy = ', (preds == y).sum() / N) 
    
    return estimator_list, estimator_weight_list, sample_weight_list

estimator_list, estimator_weight_list, sample_weight_list  = AdaBoost_scratch(X, y, M=10, learning_rate=0.001)

0.17713540534666378 1.0 0.17713540534666378
0.000217    3786
0.000218     815
Name: yesno, dtype: int64
0.1774076726113537 1.00027226726469 0.17735938345715271
0.000217    3786
0.000218     815
Name: yesno, dtype: int64
0.17768008547142053 1.0005446801247566 0.17758335934509775
0.000217    3786
0.000218     815
Name: yesno, dtype: int64
0.1779526435863148 1.0008172382396512 0.17780733263479526
0.000217    3786
0.000219     815
Name: yesno, dtype: int64
0.17822534661527817 1.0010899412686145 0.17803130295108657
0.000217    3786
0.000219     815
Name: yesno, dtype: int64
0.17849819421734497 1.0013627888706813 0.17825526991935858
0.000217    3786
0.000219     815
Name: yesno, dtype: int64
0.1787711860513438 1.0016357807046798 0.17847923316554554
0.000217    3786
0.000220     815
Name: yesno, dtype: int64
0.17904432177589946 1.0019089164292356 0.17870319231613035
0.000217    3786
0.000220     815
Name: yesno, dtype: int64
0.17931760104943445 1.0021821957027708 0.17892714699814605
0.000217 